<a href="https://colab.research.google.com/github/Falconwatch/llm_course/blob/main/HW2/%D0%94%D0%972_done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание ДЗ2.

В семинаре 5 предложите различные способы улучшения F1-score c помощью варьирования предобработки/моделей/количества обучаемых слоев в них и т.д.
**Используйте аналогичные данные и аналогичное разделение на подвыборки** (random_state зафиксирован у меня в семинаре).
В коде представьте ваш лучший подход к решению.

За задание можно получить максимум 10 баллов.

Разбалловка:
*   **Воспроизводимость и читабельность кода - 3 балла** (все воспроизвелось и все понятно для проверяющего - 3 балла; есть непонятные моменты, но все воспроизвелось - 2 балла; непонятный код и/или воспроизводится с небольшой правкой - 1 балл; непонятный код и/или ничего не воспроизвелось - 0 баллов).
*   **Технический отчет - 5 баллов** (за каждый описанный метод с полученной метрикой, при условии, что он корректный, 1 балл, соответственно для полного балла необходимо минимум 5 методов протестировать).
*   **Иновационность - 2 балла** (используете нетривиальную обработку/лосс-функцию/модель и можете объяснить свой результат - 2 балла; используете нетривиальную обработку/лосс-функцию/модель и не можете объяснить свой результат - 1 балл; просто перебираете модели из коробок - 0 баллов).

# 1. Информация о сабмите

**Щербаков Игорь Андреевич**

# 2. Технический отчет

***Введите сюда** подробное описание предложенных методов и экспериментов, с помощью которых вы пришли именно к выбору этих методов. НЕ вставляйте код в эту часть. Описание должно состоять минимум из 2-4 абзацев и содержать следующее: тип модели, параметры, как вы выбрали параметры, какие дальнейшие модификации готовых решений и т.д. вы использовали. Сюда можно включить, например, некоторые хитрости вашей предварительной обработки, описание моделей и мотивацию их использования, описание деталей процесса обучения. Если нужно, вставьте сюда графики, математические формулы.*

*Балл за «инновационность» будет присваиваться на основе содержания этой части. Пробуйте разные подходы, модели, экспериментируйте с предварительной обработкой, параметрами и т. д. Можно переопределить уже существующий подход. Это нормально, что некоторые из ваших экспериментов не сработали так, как вы ожидали. Покажите нам, что вы проявили творческий подход и провели несколько экспериментов.*

Пример решения на полный балл для похожей задачи (цифры в таблице приведены информативно):

Method | F1 Average |
--- | --- |
Baseline | 0.447 |
Rubert-tiny2 (unfreeze first layer) | 0.514 |
Rubert-tiny2 (unfreeze all layers) | 0.575 |
Distilbert (unfreeze all layers) | 0.662 |
Distilbert (unfreeze all layers, custom loss) | 0.664 |
Deberta_base | 0.740 |

Методология моей работы заключается в тестировании различных инициализаций энкодеров, сравнении дообучения нескольких слоев и всех слоев модели, настройки параметров обучения.

*   В качестве первой модели рассматривался **rubert_tiny2**,
размораживался только первый слой, обучение происходило 10 эпох с lr=2e-5. После обучения берем модель, у которой было максимальное значение F1 на выборке для валидации. Здесь не ожидались сильно высокие метрики, так как заведомо известно, что эта модель под русский язык, а датасет на английском. Но из плюсов то, что модель маленькая и быстро обучается. Значение F1 в этом эксперименте 0.514.

*   Далее бралась та же модель с теми же параметрами, но размораживались теперь все веса при обучении. Значение F1 в этом эксперименте 0.575.

*   Далее от **rubert_tiny2** переходим к **distilbert**.Размораживались все слои, обучение происходило 3 эпохи с lr=2e-5. Значение F1 в этом эксперименте 0.662.

*   Также с **distilbert** были еще попытка перевзвесить классы на обучении (минорным классам дать веса побольше, чтобы модель их лучше выучила). Обучение происходило с разморозкой всех слоев 10 эпох с lr=2e-5. Значение F1 в этом эксперименте 0.664. Как видно, модель дольше сходится к локальному минимуму, метрики остались примерно такими же.

*   Следующий подход был использовать модель **deberta_base** от Microsoft. Размораживались все слои, обучение происходило 5 эпох с lr=2e-5. Значение F1 в этом эксперименте 0.740.

Выводы:
*   Вывод 1
*   Вывод 2
*   Вывод 3


# 3. *Code*

*Введите сюда весь код, использованный для получения результатов. Добавьте несколько комментариев и подразделов для навигации по вашему решению.*

*В этой части вам предстоит самостоятельно разработать решение задачи и предоставить воспроизводимый код:*
- *Использование Python 3;*
- *Содержит код для установки всех зависимостей;*
- *Содержит код для загрузки всех используемых наборов данных*;
- *Содержит код для воспроизведения ваших результатов (другими словами, если проверяющий загрузит ваш блокнот, он сможет выполнить код по ячейкам и получить результаты эксперимента, как описано в разделе методологии)*.


*В результате ваш код будет оценен по следующим критериям:*
- ***Читаемость**: ваш код должен быть хорошо структурирован, желательно с указанием частей вашего подхода (предварительная обработка, обучение модели, тестирование модели и т. д.).*
- ***Воспроизводимость**: ваш код должен воспроизводиться без ошибок в режиме «Выполнить все» (получение экспериментальной части).*


## 3.1 Импорт библиотек

In [1]:
# some imports
!pip install -q Partial State
!pip install -q --upgrade accelerate
!pip install -q --upgrade transformers
!pip install datasets seqeval corus razdel -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.

In [2]:
from datasets import load_dataset, load_metric
from corus import load_rudrec
from razdel import tokenize
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from datasets import Dataset, DatasetDict
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification

## 3.2 Загрузка данных

### 3.2.1 Загрузка в DatasetDict



In [3]:
# download the data
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2024-07-14 11:41:50--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2024-07-14 11:41:50--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.01s   

2024-07-14 11:41:51 (124 MB/s) - ‘rudrec_annotated.json’ saved [1773014/1773014]



In [4]:
# Читаем датасет
drugs = list(load_rudrec('rudrec_annotated.json'))
print("Документов в корпусе", len(drugs))
print("Пример документа", drugs[0])
print("###############################")
print("посчитаем теги и их частоту...")
from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))


print("###############################")
print("Мэпим тэги на слова как в семинаре")
def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}
print("На примере первого текста:")
print(extract_labels(drugs[0]))

print("Обрабатываем все тексты...")
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

print("Соберём все метки в список")
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list


print("Положим всё в DatasetDict")
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data



Документов в корпусе 4809
Пример документа RuDReCRecord(file_name='172744.tsv', text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n', sentence_id=0, entities=[RuDReCEntity(entity_id='*[0]_se', entity_text='виферона', entity_type='Drugform', start=122, end=130, concept_id='C0021735', concept_name=nan), RuDReCEntity(entity_id='*[1]', entity_text='сыпью покрылся', entity_type='ADR', start=31, end=45, concept_id='C0015230', concept_name=nan), RuDReCEntity(entity_id='*[2]', entity_text='глаза опухли', entity_type='ADR', start=47, end=59, concept_id='C4760994', concept_name=nan), RuDReCEntity(entity_id='*[3]', entity_text='на веках высыпала сыпь', entity_type='ADR', start=76, end=98, concept_id='C0015230', concept_name=nan)])
###############################
посчитаем теги и их частоту...
DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профи

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3847
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 962
    })
})

### 3.2.2 Препроцессим

In [5]:
def tokenize_and_align_labels_common(examples, tokenizer, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## 3.3. Модель
Модели подсматриваю тут: https://habr.com/ru/articles/669674/

In [6]:
EPOCHS = 3

In [7]:
MODELS_DICT = dict()
def add_model(model, name):
  model_dict = dict()
  model_dict['model'] = model
  MODELS_DICT[name] = model_dict

def add_result(result, name):
  if name in MODELS_DICT:
    MODELS_DICT[name]["result"] = result
  else:
    raise BaseException("Нет модели с таким именем в списке")

In [8]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [9]:
metric = load_metric("seqeval")

<ipython-input-9-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


The repository for seqeval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/seqeval.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [10]:
def compute_metrics_common(p, tokenizer):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        for (p, l, t) in zip(prediction, label, tokens):
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### 3.3.1 Rubert Tiny2 Freezed
Модель с замороженными весами

In [11]:
rubert_tiny_checkpoint = "cointegrated/rubert-tiny2"
batch_size = 16

In [ ]:
rt_tokenizer = AutoTokenizer.from_pretrained(rubert_tiny_checkpoint)

def compute_metrics_rt(p):
  return compute_metrics_common(p, rt_tokenizer)

def tokenize_and_align_labels_rt(examples,label_all_tokens=True):
  return tokenize_and_align_labels_common(examples=examples,
                                          tokenizer=rt_tokenizer,
                                          label_all_tokens = label_all_tokens
                                          )

rt_tokenized_datasets = ner_data.map(tokenize_and_align_labels_rt, batched=True)
rt_data_collator = DataCollatorForTokenClassification(rt_tokenizer)

Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

{'tokens': [['По', 'мне', ',', 'так', 'это', 'пустышка', ',', 'которая', 'может', 'обладать', 'только', 'эффектом', 'плацебо', '.'], ['Упаковка', 'красивая', ',', 'бросается', 'в', 'глаза', '.'], ['Мой', 'мозг', 'не', 'может', 'постичь', 'творческий', 'полет', 'мыслей', 'создателей', 'сего', 'сиропа', '.'], ['Препарат', 'нам', 'подошел', 'и', 'очень', 'помог', ':', 'если', 'ребенок', 'раньше', 'болел', 'до', '10', 'раз', 'за', 'год', ',', 'то', 'после', 'нескольких', 'курсов', '1-2', 'раза', 'в', 'год', '!'], ['Таблеточки', 'не', 'приносят', 'никакого', 'неудобства', ',', 'их', 'просто', 'нужно', 'проглотить', ')', 'Раньше', 'совсем', 'таблетки', 'не', 'пила', ',', 'а', 'тут', 'подхватила', 'вирус', 'гриппа', 'и', 'разболелась', '.'], ['Мы', 'поменяли', 'место', 'жительства', 'и', 'перевели', 'дочь', 'в', 'школу', ',', 'которая', 'находится', 'ближе', 'к', 'дому', '.'], ['Я', 'работаю', 'в', 'аптеке', 'и', 'мне', 'как', 'бонус', 'подарили', 'препарат', 'под', 'названием', 'Инфлюцид', '

In [13]:
rubert_tiny_freezed = AutoModelForTokenClassification.from_pretrained(rubert_tiny_checkpoint,
                                                                      num_labels=len(label_list),
                                                                      device_map = 'cuda')
rubert_tiny_freezed.config.id2label = dict(enumerate(label_list))
rubert_tiny_freezed.config.label2id = {v: k for k, v in rubert_tiny_freezed.config.id2label.items()}


for param in rubert_tiny_freezed.bert.parameters():
    param.requires_grad = False

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
rtf_args = TrainingArguments(
    "ner",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

# model training
rtf_trainer = Trainer(
    rubert_tiny_freezed,
    rtf_args,
    train_dataset=rt_tokenized_datasets["train"],
    eval_dataset=rt_tokenized_datasets["test"],
    data_collator=rt_data_collator,
    tokenizer=rt_tokenizer,
    compute_metrics=compute_metrics_rt
)

In [15]:
rtf_trainer.evaluate()
rtf_trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.820062,0.015684,0.026427,0.019685,0.805792
2,No log,1.550548,0.011940,0.004228,0.006245,0.881658
3,1.859400,1.466446,0.020513,0.004228,0.007011,0.889857


TrainOutput(global_step=723, training_loss=1.7493413183863893, metrics={'train_runtime': 12.6422, 'train_samples_per_second': 912.894, 'train_steps_per_second': 57.189, 'total_flos': 7436491614990.0, 'train_loss': 1.7493413183863893, 'epoch': 3.0})

In [16]:
rtf_predictions = rtf_trainer.predict(rt_tokenized_datasets["test"])

add_model(rubert_tiny_freezed, "rubert_tiny_freezed")
add_result(rtf_predictions.metrics, "rubert_tiny_freezed")

### 3.3.2 Rubert Tiny2 Unfreezed
Модель с размороженными весами

In [17]:
rubert_tiny_unfreezed = AutoModelForTokenClassification.from_pretrained(rubert_tiny_checkpoint,
                                                                        num_labels=len(label_list),
                                                                        device_map = 'cuda')
rubert_tiny_unfreezed.config.id2label = dict(enumerate(label_list))
rubert_tiny_unfreezed.config.label2id = {v: k for k, v in rubert_tiny_unfreezed.config.id2label.items()}


for param in rubert_tiny_unfreezed.bert.parameters():
    param.requires_grad = True

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
rtuf_args = TrainingArguments(
    "ner",
    eval_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

rtuf_trainer = Trainer(
    rubert_tiny_unfreezed,
    rtuf_args,
    train_dataset=rt_tokenized_datasets["train"],
    eval_dataset=rt_tokenized_datasets["test"],
    data_collator=rt_data_collator,
    tokenizer=rt_tokenizer,
    compute_metrics=compute_metrics_rt
)

In [19]:
rtuf_trainer.evaluate()
rtuf_trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.428481,0.743590,0.030655,0.058883,0.902933
2,No log,0.356422,0.549261,0.235729,0.329882,0.916303
3,0.629500,0.343659,0.468124,0.271670,0.343813,0.917781


TrainOutput(global_step=723, training_loss=0.5374376605655148, metrics={'train_runtime': 23.8335, 'train_samples_per_second': 484.235, 'train_steps_per_second': 30.335, 'total_flos': 7436491614990.0, 'train_loss': 0.5374376605655148, 'epoch': 3.0})

In [20]:
rtuf_predictions = rtuf_trainer.predict(rt_tokenized_datasets["test"])

add_model(rubert_tiny_unfreezed, "rubert_tiny_unfreezed")
add_result(rtuf_predictions.metrics, "rubert_tiny_unfreezed")

### 3.3.2-2 RT2 Unfreezed + all labled

In [149]:
rubert_tiny_unfreezed_labeled = AutoModelForTokenClassification.from_pretrained(rubert_tiny_checkpoint,
                                                                        num_labels=len(label_list),
                                                                        device_map = 'cuda')
rubert_tiny_unfreezed_labeled.config.id2label = dict(enumerate(label_list))
rubert_tiny_unfreezed_labeled.config.label2id = {v: k for k, v in rubert_tiny_unfreezed_labeled.config.id2label.items()}


for param in rubert_tiny_unfreezed_labeled.bert.parameters():
    param.requires_grad = True

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [154]:
# Делаю вызов токенищации, лейблы ставлю всем токенам
def tokenize_and_align_labels_rt_all(examples):
  return tokenize_and_align_labels_common(examples=examples,
                                          tokenizer=rt_tokenizer,
                                          label_all_tokens = True
                                          )

In [155]:
rt_tokenized_datasets_al = ner_data.map(tokenize_and_align_labels_rt_all, batched=True)

Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

In [157]:
rtuf_trainer_al = Trainer(
    rubert_tiny_unfreezed,
    rtuf_args,
    train_dataset=rt_tokenized_datasets_al["train"],
    eval_dataset=rt_tokenized_datasets_al["test"],
    data_collator=rt_data_collator,
    tokenizer=rt_tokenizer,
    compute_metrics=compute_metrics_rt
)

In [ ]:
rtuf_trainer_al.evaluate()
rtuf_trainer_al.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.416823,0.443750,0.367876,0.402266,0.922820
2,No log,0.390807,0.463647,0.403109,0.431264,0.925502


### 3.3.3 T5 small

In [21]:
t5_checkpoint = "cointegrated/rut5-small"
batch_size = 16

In [22]:
t5_tokenizer = AutoTokenizer.from_pretrained(t5_checkpoint)

def compute_metrics_t5(p):
  return compute_metrics_common(p, t5_tokenizer)

def tokenize_and_align_labels_t5(examples,label_all_tokens=False):
  return tokenize_and_align_labels_common(examples=examples,
                                          tokenizer=t5_tokenizer,
                                          )

t5_tokenized_datasets = ner_data.map(tokenize_and_align_labels_t5, batched=True)
t5_data_collator = DataCollatorForTokenClassification(t5_tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/640k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/962 [00:00<?, ? examples/s]

In [23]:
t5 = AutoModelForTokenClassification.from_pretrained(t5_checkpoint,
                                                       num_labels=len(label_list),
                                                       device_map="cuda"
                                 )

t5.config.id2label = dict(enumerate(label_list))
t5.config.label2id = {v: k for k, v in t5.config.id2label.items()}

for param in t5.modules():
    param.requires_grad = True

model.safetensors:   0%|          | 0.00/259M [00:00<?, ?B/s]

Some weights of MT5ForTokenClassification were not initialized from the model checkpoint at cointegrated/rut5-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
t5_args = TrainingArguments(
    "ner",
    eval_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

t5_trainer = Trainer(
    t5,
    t5_args,
    train_dataset=t5_tokenized_datasets["train"],
    eval_dataset=t5_tokenized_datasets["test"],
    data_collator=t5_data_collator,
    tokenizer=t5_tokenizer,
    compute_metrics=compute_metrics_t5
)

In [26]:
t5_trainer.evaluate()
t5_trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.266410,0.086134,0.043340,0.057665,0.882618
2,No log,0.933563,0.167897,0.096195,0.122312,0.892000
3,2.570600,0.853328,0.190705,0.125793,0.151592,0.893920


TrainOutput(global_step=723, training_loss=2.0889432915018804, metrics={'train_runtime': 70.8046, 'train_samples_per_second': 162.998, 'train_steps_per_second': 10.211, 'total_flos': 91668135124800.0, 'train_loss': 2.0889432915018804, 'epoch': 3.0})

In [27]:
t5_predictions = t5_trainer.predict(t5_tokenized_datasets["test"])

add_model(t5, "rut5-small")
add_result(t5_predictions.metrics, "rut5-small")

### 3.3.4 T5 Small freezed

In [28]:
t5_freezed = AutoModelForTokenClassification.from_pretrained(t5_checkpoint,
                                                       num_labels=len(label_list),
                                                       device_map="cuda"
                                 )

t5_freezed.config.id2label = dict(enumerate(label_list))
t5_freezed.config.label2id = {v: k for k, v in t5_freezed.config.id2label.items()}

for param in t5_freezed.modules():
    param.requires_grad = False

Some weights of MT5ForTokenClassification were not initialized from the model checkpoint at cointegrated/rut5-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
t5_freezed

MT5ForTokenClassification(
  (transformer): MT5EncoderModel(
    (shared): Embedding(20100, 512)
    (encoder): MT5Stack(
      (embed_tokens): Embedding(20100, 512)
      (block): ModuleList(
        (0): MT5Block(
          (layer): ModuleList(
            (0): MT5LayerSelfAttention(
              (SelfAttention): MT5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): MT5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): MT5LayerFF(
              (DenseReluDense): MT5DenseGatedActDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (

In [29]:
t5f_trainer = Trainer(
    t5_freezed,
    t5_args,
    train_dataset=t5_tokenized_datasets["train"],
    eval_dataset=t5_tokenized_datasets["test"],
    data_collator=t5_data_collator,
    tokenizer=t5_tokenizer,
    compute_metrics=compute_metrics_t5
)

In [30]:
t5f_trainer.evaluate()
t5f_trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.266410,0.086134,0.043340,0.057665,0.882618
2,No log,0.933563,0.167897,0.096195,0.122312,0.892000
3,2.570600,0.853328,0.190705,0.125793,0.151592,0.893920


TrainOutput(global_step=723, training_loss=2.0889432915018804, metrics={'train_runtime': 56.2336, 'train_samples_per_second': 205.233, 'train_steps_per_second': 12.857, 'total_flos': 91668135124800.0, 'train_loss': 2.0889432915018804, 'epoch': 3.0})

In [31]:
t5f_predictions = t5f_trainer.predict(t5_tokenized_datasets["test"])

add_model(t5_freezed, "rut5-small-freezed")
add_result(t5f_predictions.metrics, "rut5-small-freezed")

## 3.4. Применение модели

In [32]:
# model inference

In [33]:
results = list()
for k in MODELS_DICT.keys():
  tmp = MODELS_DICT[k]["result"]
  results.append(pd.DataFrame.from_records(tmp, index=[k]))

results_table = pd.concat(results)

In [34]:
results_table

,test_accuracy,test_f1,test_loss,test_precision,test_recall,test_runtime,test_samples_per_second,test_steps_per_second
rubert_tiny_freezed,0.889857,0.007011,1.466446,0.020513,0.004228,0.8433,1140.763,72.335
rubert_tiny_unfreezed,0.917781,0.343813,0.343659,0.468124,0.271670,0.9254,1039.571,65.919
rut5-small,0.893920,0.151592,0.853328,0.190705,0.125793,1.7402,552.807,35.053
rut5-small-freezed,0.893920,0.151592,0.853328,0.190705,0.125793,1.7955,535.772,33.973
